<a href="https://colab.research.google.com/github/Rotichkiprono/NeuralNetworks/blob/main/CarorTruck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing image data for binary classification

In [6]:
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory  #for loading images from a directory and creating a dataset.

# ensures the same random results each time the code runs.
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells


# Load training and validation sets
ds_train_ = image_dataset_from_directory(
    '/content/drive/MyDrive/TruckandCar/train',
    labels='inferred',  #Automatically infers labels from subdirectory names
    label_mode='binary', #Treats the classification as a binary problem
    image_size=[128, 128],
    interpolation='nearest',  # resizing images
    batch_size=64,  #Groups images for faster processing during training.
    shuffle=True,   #Randomly shuffles the training data for better generalization.
)
ds_valid_ = image_dataset_from_directory(
    '/content/drive/MyDrive/TruckandCar/valid',
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=False, # shuffling is best used in training only
)

# Data Pipeline Converts the image data type from potentially integer to float32 for better compatibility
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float) # applying the conversion function and other techniques
    .cache()                # caches the processed data to improve performance on subsequent iterations.
    .prefetch(buffer_size=AUTOTUNE) #
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


Found 5117 files belonging to 2 classes.
Found 5051 files belonging to 2 classes.


# Defining a model

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    # Block One
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same',
                  input_shape=[128, 128, 3]),
    layers.MaxPool2D(), #reduce the dimensionality of the data and capture spatial features.

    # Block Two
    layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Block Three
    layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Head
    layers.Flatten(),  #helps in reducing the number of parameters in the subsequent layers
    layers.Dense(6, activation='relu'), #performs some high-level feature extraction from the flattened data.
    layers.Dropout(0.2),      # prevent overfitting by randomly dropping 20% of the neurons during training.
    layers.Dense(1, activation='sigmoid'), #outputs a value between 0 and 1, truck(closer to 1) car (closer to 0)
])

# Defining the architecture

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(epsilon=0.01),
    loss = 'binary_crossentropy',
    metrics = ['binary_accuracy']
)

# Fitting the training set

In [9]:
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=50,
)

Epoch 1/50


KeyboardInterrupt: 

# plotting

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();
